# Data Processing

## Goals

This program aims to:
- Get us a brief looking of the missing data
- Deal with the missing data
- Transform data into suitable integer format for numpy array

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from conversion import *
from extract_postal import extract
from get_room import get_room

In [2]:
# there seems to be some ASCII character encoded in the raw data file
# in order to make pandas able to read the file, I added 'encoding = "unicode_escape"'

df = pd.read_csv("data_raw.csv", encoding = "unicode_escape")

df.head(5)

,Quarter,Postal code,Building type,Price per square meter (EUR/m2)
0,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, one-room flat",5458
1,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, two-room flat",5164
2,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, three-room flat+",4944
3,2010Q1,00120 Punavuori (Helsinki ),"Blocks of flats, one-room flat",5515
4,2010Q1,00120 Punavuori (Helsinki ),"Blocks of flats, two-room flat",5349


In [3]:
# change the column names
df.columns = ["quarter", "address", "type", "price"]

# convert the rows with price value missing into NaN value.
df.loc[(df.price == ".."), "price"] = np.NaN

"""
Here, I want to extract the information from 3 features [quarter, address, type] into a numerical list
so that we can use it for machine learning model.

These functions are written in the external files conversion.py, extract_postal.py, and get_room.py.

In the end, we insert new columns into our dataframe.
"""

date_count = conversion()
postal_code, city = extract()
room = get_room()

df["date_count"] = date_count
df["postal_code"] = postal_code
df["city"] = city
df["room"] = room

# change the type of the postal_code column into integer
df["postal_code"] = df["postal_code"].astype(int)

df

,quarter,address,type,price,date_count,postal_code,city,room
0,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, one-room flat",5458,1,100,Helsinki,1
1,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, two-room flat",5164,1,100,Helsinki,2
2,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, three-room flat+",4944,1,100,Helsinki,3
3,2010Q1,00120 Punavuori (Helsinki ),"Blocks of flats, one-room flat",5515,1,120,Helsinki,1
4,2010Q1,00120 Punavuori (Helsinki ),"Blocks of flats, two-room flat",5349,1,120,Helsinki,2
...,...,...,...,...,...,...,...,...
23602,2021Q4*,02970 Kalajärvi (Espoo ),"Blocks of flats, two-room flat",NaN,48,2970,Espoo,2
23603,2021Q4*,02970 Kalajärvi (Espoo ),"Blocks of flats, three-room flat+",NaN,48,2970,Espoo,3
23604,2021Q4*,02980 Lakisto (Espoo ),"Blocks of flats, one-room flat",NaN,48,2980,Espoo,1
23605,2021Q4*,02980 Lakisto (Espoo ),"Blocks of flats, two-room flat",NaN,48,2980,Espoo,2


In [4]:
print(df.shape)
print(df["price"].isna().sum())

(23607, 8)
16127


In [5]:
"""
From the cell above, we can see that 16127 out of 23607 rows have missing price.
It is indeed a big portion of the data. However, if we just ignore these rows,
we still have 7000 datapoints, and it is enough for our machine learning model.
"""

# remove rows which have missing values
df = df.dropna(axis=0)

# reset the row indices 
df = df.reset_index()

df

,index,quarter,address,type,price,date_count,postal_code,city,room
0,0,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, one-room flat",5458,1,100,Helsinki,1
1,1,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, two-room flat",5164,1,100,Helsinki,2
2,2,2010Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),"Blocks of flats, three-room flat+",4944,1,100,Helsinki,3
3,3,2010Q1,00120 Punavuori (Helsinki ),"Blocks of flats, one-room flat",5515,1,120,Helsinki,1
4,4,2010Q1,00120 Punavuori (Helsinki ),"Blocks of flats, two-room flat",5349,1,120,Helsinki,2
...,...,...,...,...,...,...,...,...,...
7475,23551,2021Q4*,02650 Pohjois-Leppävaara (Espoo ),"Blocks of flats, two-room flat",4564,48,2650,Espoo,2
7476,23552,2021Q4*,02650 Pohjois-Leppävaara (Espoo ),"Blocks of flats, three-room flat+",3929,48,2650,Espoo,3
7477,23561,2021Q4*,02700 Kauniainen (Kauniainen ),"Blocks of flats, three-room flat+",4824,48,2700,Kauniainen,3
7478,23564,2021Q4*,02710 Viherlaakso (Espoo ),"Blocks of flats, three-room flat+",2782,48,2710,Espoo,3


In [6]:
# remove column "quarter", "address", "type", "index" that are not used
df = df.drop(["quarter", "address", "type", "index"], axis=1)

# switch the column order
df = df[["city", "date_count", "postal_code", "room", "price"]]

df

,city,date_count,postal_code,room,price
0,Helsinki,1,100,1,5458
1,Helsinki,1,100,2,5164
2,Helsinki,1,100,3,4944
3,Helsinki,1,120,1,5515
4,Helsinki,1,120,2,5349
...,...,...,...,...,...
7475,Espoo,48,2650,2,4564
7476,Espoo,48,2650,3,3929
7477,Kauniainen,48,2700,3,4824
7478,Espoo,48,2710,3,2782


In [7]:
# export the processed file to a new csv file
# we will the exported file for our machine learning model

df.to_csv("data_cleaned.csv", index=False)